<a href="https://colab.research.google.com/github/Suraj209211/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/blob/main/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Step 1: Load the dataset
data_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/NEW_DATA/CSV_OUTPUTS/finaldata_geophysicalchemical.csv'  # Update this path
data = pd.read_csv(data_path)

In [ ]:
features = data[['X', 'Y', 'GravValue', 'MagneticValue', 'SiO2', 'Al2O3', 'Fe2O3', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'TiO2', 'P2O5']]
target = data['PROBABILITY']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Step 4: Normalize the feature data for better model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Step 5: Build a deep learning model
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification (presence or absence of mineral)
])

# Step 6: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
# Step 9: Visualize training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# Step 10: Generate predictions and visualize them based on lat-long values
# Make predictions on the test data
y_pred = model.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary (0 or 1)

# Visualize predictions
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_test['X'], y=X_test['Y'], hue=y_pred_binary[:,0], palette='coolwarm', legend='full')
plt.title('Predicted Mineral Presence Based on Latitude and Longitude')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Step 11: Visualize specific minerals from the dataset
# Visualizing distribution of a specific mineral like Fe2O3 (Iron oxide)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=data['X'], y=data['Y'], hue=data['Fe2O3'], palette='viridis')
plt.title('Geographical Distribution of Fe2O3 (Iron Oxide)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Step 10: Generate predictions
y_pred_prob = model.predict(X_test_scaled)  # Get prediction probabilities
y_pred_binary = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary (0 or 1)

# Combine the predictions with the test data's latitude and longitude for heatmap plotting
heatmap_data = pd.DataFrame({
    'Longitude': X_test['X'],
    'Latitude': X_test['Y'],
    'Prediction_Probability': y_pred_prob.flatten(),  # Use probabilities for heatmap
    'Prediction_Binary': y_pred_binary.flatten()      # Or use binary predictions (0/1)
})

# Step 11: Generate a heatmap
# We will bin the Latitude and Longitude into grids for the heatmap
heatmap_data_binned = heatmap_data.pivot_table(index='Latitude', columns='Longitude', values='Prediction_Probability')

# Generate the heatmap
plt.figure(figsize=(30, 20))
sns.heatmap(heatmap_data_binned, cmap='coolwarm', annot=False, cbar=True)
plt.title('Heatmap of Mineral Prediction Probabilities')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming the model has been trained as per the earlier steps
# and the dataset is loaded into 'data'

# Step 1: Prepare the features for the model (using the same features from before)
features = data[['X', 'Y', 'GravValue', 'MagneticValue', 'SiO2', 'Al2O3', 'Fe2O3', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'TiO2', 'P2O5']]
target = data['PROBABILITY']  # Assuming this is the target variable indicating mineral presence

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Step 3: Normalize the feature data for better model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Generate predictions
y_pred_prob = model.predict(X_test_scaled)  # Get prediction probabilities

# Step 5: Prepare the data for the heatmap
heatmap_data = pd.DataFrame({
    'Longitude': X_test['X'],
    'Latitude': X_test['Y'],
    'Prediction_Probability': y_pred_prob.flatten()  # Use prediction probabilities
})

# Step 6: Create a pivot table to arrange the data by Latitude and Longitude
heatmap_data_binned = heatmap_data.pivot_table(index='Latitude', columns='Longitude', values='Prediction_Probability')

# Step 7: Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(heatmap_data_binned, cmap='coolwarm', annot=False, cbar=True)
plt.title('Mineral Prediction Heatmap Based on Model')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# After training the model, save it to a file
model_save_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/MODEL/DNN.h5'  # Update this path to where you'd like to save the model
model.save(model_save_path)

print(f"Model saved to {model_save_path}")